# Train YOLOv1

In [ ]:
import shutil
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('../')
from torchsummary.torchsummary import summary
from dataloader import VOC
import numpy as np
import matplotlib.pyplot as plt
import yolov1
from yolov1 import detection_collate
from yolov1 import detection_loss
from yolov1 import save_checkpoint

In [ ]:
# Convolutional neural network (two convolutional layers)
class YOLOv1(nn.Module):
    def __init__(self):
        super(YOLOv1, self).__init__()
        # LAYER 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64), #, momentum=0.01
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(192, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(192, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128, momentum=0.01),
            nn.LeakyReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 4
        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256, momentum=0.01),
            nn.LeakyReLU())
        self.layer14 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer15 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer16 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # LAYER 5
        self.layer17 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer18 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer19 = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(512, momentum=0.01),
            nn.LeakyReLU())
        self.layer20 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer21 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer22 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())

        # LAYER 6
        self.layer23 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())
        self.layer24 = nn.Sequential(
            nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024, momentum=0.01),
            nn.LeakyReLU())

        self.fc1 = nn.Sequential(
            nn.Linear(7*7*1024, 4096),
            nn.LeakyReLU(),
            nn.Dropout(dropout_prop)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 7*7*((5)+num_classes)),
            nn.Dropout(dropout_prop),
        )


        for m in self.modules():

            if isinstance(m, nn.Conv2d):
                #nn.init.xavier_normal_(m.weight)
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity="leaky_relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        out = self.layer17(out)
        out = self.layer18(out)
        out = self.layer19(out)
        out = self.layer20(out)
        out = self.layer21(out)
        out = self.layer22(out)
        out = self.layer23(out)
        out = self.layer24(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = out.reshape((-1,7,7,((5)+num_classes)))

        return out


In [ ]:
def detection_loss_test(output, target):
    # hyper parameter

    lambda_coord = 5
    lambda_noobj = 0.5

    # check batch size
    b, _, _, _ = target.shape

    # class loss
    MSE_criterion = nn.MSELoss()

    # output tensor slice
    objness1_output = output[:, :, :, 0]
    x_offset1_output = output[:, :, :, 1]
    y_offset1_output = output[:, :, :, 2]
    width_ratio1_output = output[:, :, :, 3]
    height_ratio1_output = output[:, :, :, 4]
    class_output = output[:, :, :, 5]

    # label tensor slice
    objness_label = target[:, :, :, 0]
    class_label = target[:, :, :, 1]
    x_offset_label = target[:, :, :, 2]
    y_offset_label = target[:, :, :, 3]
    width_ratio_label = target[:, :, :, 4]
    height_ratio_label = target[:, :, :, 5]
    
    noobjness_label = torch.neg(torch.add(objness_label, -1))
    #y_one_hot = one_hot(class_output, class_label).cuda()

    #print(y_one_hot)
    #print(y_one_hot.shape)
    #exit()

    obj_coord1_loss = lambda_coord * \
                      torch.sum(objness_label *
                                (torch.pow(x_offset1_output - x_offset_label, 2) +
                                 torch.pow(y_offset1_output - y_offset_label, 2)))

    obj_size1_loss = lambda_coord * \
                     torch.sum(objness_label *
                               (torch.pow(width_ratio1_output - torch.sqrt(width_ratio_label), 2) +
                                torch.pow(height_ratio1_output - torch.sqrt(height_ratio_label), 2)))

    #obj_coord2_loss = lambda_coord * \
    #                  torch.sum(objness_label *
    #                            (torch.pow(x_offset2_output - x_offset_label, 2) +
    #                             torch.pow(y_offset2_output - y_offset_label, 2)))

    #obj_size2_loss = lambda_coord * \
    #                 torch.sum(objness_label *
    #                           (torch.pow(width_ratio2_output - torch.sqrt(width_ratio_label), 2) +
    #                            torch.pow(height_ratio2_output - torch.sqrt(height_ratio_label), 2)))



    obj_class_loss = torch.sum(objness_label * torch.pow(class_output - class_label, 2))

    #noobj_class_loss = lambda_noobj * torch.sum(noobjness_label * MSE_criterion(class_output, y_one_hot))
    
    noobjness1_loss = lambda_noobj * torch.sum(noobjness_label * torch.pow(objness1_output - objness_label, 2))
    #noobjness2_loss = lambda_noobj * torch.sum(objness_label * torch.pow(objness2_output - objness_label, 2))
    
    objness1_loss = torch.sum(objness_label * torch.pow(objness1_output - objness_label, 2))
    #objness2_loss = torch.sum(objness_label * torch.pow(objness2_output - objness_label, 2))

    #total_loss = (obj_coord1_loss + obj_size1_loss + obj_coord2_loss + obj_size2_loss + noobjness1_loss + noobjness2_loss +
    #              obj_class_loss + objness1_loss + objness2_loss) / b
    total_loss = (obj_coord1_loss + obj_size1_loss + noobjness1_loss  + objness1_loss + obj_class_loss) / b

    #return total_loss, obj_coord1_loss, obj_size1_loss, obj_coord2_loss, obj_size2_loss, obj_class_loss, noobjness1_loss, noobjness2_loss, objness1_loss, objness2_loss
    return total_loss, obj_coord1_loss / b, obj_size1_loss/ b, obj_class_loss/ b, noobjness1_loss/ b, objness1_loss/ b

# 0. Vidom on

In [ ]:
def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 1)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )

In [ ]:
def update_vis_plot(iteration, loss, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 1)).cpu() * iteration,
        Y=torch.Tensor([loss]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )

In [ ]:
import visdom
viz = visdom.Visdom()

vis_title = 'Yolo V1 Deepbaksu_vision (feat. martin, visionNoob) PyTorch on ' + 'VOC'
vis_legend = ['Train Loss']

iter_plot = yolov1.create_vis_plot(viz, 'Iteration', 'Total Loss', vis_title, vis_legend)

coord1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'coord1', vis_title, vis_legend)
size1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'size1', vis_title, vis_legend)

#coord2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'coord2', vis_title, vis_legend)
#size2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'size2', vis_title, vis_legend)

noobjectness1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobjectness1', vis_title, vis_legend)
#noobjectness2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobjectness2', vis_title, vis_legend)

objectness1_plot = yolov1.create_vis_plot(viz, 'Iteration', 'objectness1', vis_title, vis_legend)
#objectness2_plot = yolov1.create_vis_plot(viz, 'Iteration', 'objectness2', vis_title, vis_legend)

obj_cls_plot = yolov1.create_vis_plot(viz, 'Iteration', 'obj_cls', vis_title, vis_legend)
#noobj_cls_plot = yolov1.create_vis_plot(viz, 'Iteration', 'noobj_cls', vis_title, vis_legend)

# 1. Device configuration

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 16000
num_classes = 1
batch_size = 15
learning_rate = 1e-3

dropout_prop = 0.5

# 2. Load VOC Pascal'12 Dataset

In [ ]:
DATASET_PATH_MARTIN = "/media/keti-ai/AI_HARD3/DataSets/VOC_Pascal/VOC/VOCdevkit/VOC2012"
DATASET_PATH_JAEWON = "H:\VOC\VOC12\VOCdevkit_2\VOC2012"
SMALL_DATASET_PATH = "D:/dataset/person-300"
train_dataset = VOC(root = SMALL_DATASET_PATH,
                    transform=transforms.ToTensor(), cls_option = True, selective_cls="person")

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

# 3. Load YOLOv1

In [ ]:
net = yolov1.YOLOv1()
# visualize_weights_distribution(net)
use_cuda = torch.cuda.is_available()

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")
model = YOLOv1().to(device)
#model = torch.nn.DataParallel(net, device_ids=[0]).cuda()

summary(model, (3, 448,448))

# Sanity Check for output dimention

In [ ]:
images, labels = iter(train_loader).next()
images = images.to(device)
labels = labels.to(device)

In [ ]:
#for just a image
num = 2
test_image = images[num]
outputs = model(torch.cuda.FloatTensor(np.expand_dims(test_image,axis=0)))
print(outputs.shape)

#for images (batch size)
outputs = model(torch.cuda.FloatTensor(images))
print(outputs.shape)

In [ ]:
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

In [ ]:
torch.optim.Adam?

# 4.Train the model

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):

    if (epoch == 200) or (epoch == 400) or (epoch == 600) or (epoch == 20000) or (epoch == 30000):
        scheduler.step()

    for i, (images, labels) in enumerate(train_loader):

    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calc Loss
        loss, \
        obj_coord1_loss, \
        obj_size1_loss, \
        obj_class_loss, \
        noobjness1_loss, \
        objness1_loss = detection_loss_test(outputs, labels)
        #objness2_loss = yolov1.detection_loss(outputs, labels)
        
        
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            
            print('Epoch ,[{}/{}] ,Step ,[{}/{}] ,lr ,{} ,total_loss ,{:.4f} ,coord1 ,{} ,size1 ,{} ,noobj_clss ,{} ,objness1 ,{} ,'
                  .format(epoch + 1,
                          num_epochs,
                          i + 1,
                          total_step,
                          [param_group['lr'] for param_group in optimizer.param_groups],
                          loss.item(),
                          obj_coord1_loss,
                          obj_size1_loss,
                          obj_class_loss,
                          noobjness1_loss,
                          objness1_loss
                          ))
            

            yolov1.update_vis_plot(viz, (epoch+1)*batch_size +(i + 1), loss.item(), iter_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_coord1_loss, coord1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_size1_loss, size1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), obj_class_loss, obj_cls_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), noobjness1_loss, noobjectness1_plot, None, 'append')
            yolov1.update_vis_plot(viz, (epoch + 1) * batch_size + (i + 1), objness1_loss, objectness1_plot, None, 'append')


            

    if (epoch % 300) == 0:
        yolov1.save_checkpoint({
            'epoch': epoch + 1,
            'arch': "YOLOv1",
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, False, filename='checkpoint_{}.pth.tar'.format(epoch))


In [ ]:
labels[2]

In [ ]:
outputs = model(images)

In [ ]:
class_output = outputs[:, :, :, 10:]
class_label = labels[:, :, :, 1]
y_one_hot = one_hot(class_output, class_label)
y_one_hot[0,:,:,0]

In [ ]:
print(class_output[0])
print(class_label[0])

In [ ]:
class_label[0]

In [ ]:
class_label/shape

In [ ]:
objness_label = labels[:, :, :, 0]
noobjness_label = torch.neg(torch.add(objness_label, -1))

In [ ]:
labels.shape

In [ ]:
noobjness_label[0]

In [ ]:
labels[0,:,:,0]

In [ ]:
a = 

In [ ]:
outputs = model(images)

In [ ]:
# Calc Loss
loss, \
obj_coord1_loss, \
obj_size1_loss, \
obj_class_loss, \
noobj_class_loss, \
objness1_loss = detection_loss_test(outputs, labels)
    

In [ ]:
print("total loss:\t\t",loss)
print("obj_coord1_loss:\t",obj_coord1_loss)
print("obj_size1_loss:\t\t",obj_size1_loss)
print("obj_class_loss:\t\t",obj_class_loss)
print("noobj_class_loss:\t",noobj_class_loss)
print("objness1_loss:\t\t",objness1_loss)

In [ ]:
width_ratio1_output = outputs[:, :, :, 3]

In [ ]:
a = torch.sqrt(width_ratio1_output)

In [ ]:
print(width_ratio1_output)

# TEST

In [ ]:
# Test the model

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           collate_fn=detection_collate)

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
predicted.shape

In [ ]:
labels.shape

In [ ]:
outputs[0].shape

In [ ]:
images.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline 

In [ ]:
outputs[0].shape

In [ ]:
outputs[0][0][0]

In [ ]:
print(outputs[0, :, :, 0])
print(outputs[0, :, :, 5])

In [ ]:
print(outputs[0,0,0,:])
print(outputs[1,0,0,:])
print(outputs[2,0,0,:])

In [ ]:
plt.imshow(np.transpose(images[num],(1,2,0)))
plt.show()
plt.imshow(outputs.detach().cpu().numpy()[0,:, :, 0])
plt.show()
plt.imshow(outputs.detach().cpu().numpy()[0,:, :, 5])
plt.show()

In [ ]:
image.shape

In [ ]:
np.expand_dims(test_image,axis=0).shape